In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib
from xgboost import XGBClassifier

In [12]:
data = pd.read_csv('neo.csv')
data

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249220,54839744.08,Earth,False,16.73,False
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726660,61438126.52,Earth,False,20.00,True
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692100,49798724.94,Earth,False,17.83,False
3,3596030,(2012 BV13),0.096506,0.215794,24764.303140,25434972.72,Earth,False,22.20,False
4,3667127,(2014 GE35),0.255009,0.570217,42737.733760,46275567.00,Earth,False,20.09,True
...,...,...,...,...,...,...,...,...,...,...
90831,3763337,(2016 VX1),0.026580,0.059435,52078.886690,12300389.18,Earth,False,25.00,False
90832,3837603,(2019 AD3),0.016771,0.037501,46114.605070,54321206.42,Earth,False,26.00,False
90833,54017201,(2020 JP3),0.031956,0.071456,7566.807732,28400768.16,Earth,False,24.60,False
90834,54115824,(2021 CN5),0.007321,0.016370,69199.154480,68692060.53,Earth,False,27.80,False


In [13]:
X = data[['est_diameter_min', 'est_diameter_max', 'relative_velocity', 'absolute_magnitude', 'miss_distance']]
y = data['hazardous']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

C:\Users\dilip\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\dilip\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\dilip\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\dilip\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

In [16]:
xgb_model = XGBClassifier(scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train), random_state=42)
xgb_model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [17]:
y_pred = xgb_model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {np.round(accuracy * 100, 2)}%')


              precision    recall  f1-score   support

       False       0.99      0.80      0.89     16400
        True       0.34      0.93      0.49      1768

    accuracy                           0.81     18168
   macro avg       0.66      0.87      0.69     18168
weighted avg       0.93      0.81      0.85     18168

Accuracy: 81.42%


In [18]:
joblib.dump(xgb_model, 'asteroid_hazard_xgb_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [19]:
loaded_model = joblib.load('asteroid_hazard_xgb_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')

In [20]:
# Predict with new input data
input_data = [[0.265800, 0.594347, 73588.72666, 20.00, 6.143813e+07]]
input_data_scaled = loaded_scaler.transform(input_data)
prediction = loaded_model.predict(input_data_scaled)

# Map numerical predictions to True/False
prediction_label = 'True' if prediction[0] == 1 else 'False'
print(f'Prediction: {prediction_label}')


Prediction: True


C:\Users\dilip\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
